In [1]:
import datafaucet as dfc

In [2]:
# start the engine
project = dfc.project.load()

NOTICE:datafaucet:hyperloglog.ipynb:engine:set_submit_args | Configuring packages:
NOTICE:datafaucet:hyperloglog.ipynb:engine:set_submit_args |   -  mysql:mysql-connector-java:8.0.12
NOTICE:datafaucet:hyperloglog.ipynb:engine:__init__ | Connecting to spark master: local[*]
NOTICE:datafaucet:hyperloglog.ipynb:engine:__init__ | Engine context spark:2.4.4 successfully started


In [3]:
from pyspark.sql import functions as F

In [71]:
SIZE = 10_000
GROUPS = 5

In [72]:
df = dfc.range(SIZE)

In [100]:
df = (df
    .cols.create('g').randint(0, GROUPS)
    .cols.create('v').randint(0,SIZE)
)

In [101]:
df.data.grid(5)

,id,g,v
0,0,3,2361
1,1,2,5659
2,2,3,9830
3,3,1,5020
4,4,0,3487


In [102]:
(df
    .cols.get('v').agg('distinct')
).data.grid()

,v
0,6320


In [103]:
(df
    .cols.get('v').agg(F.approx_count_distinct)
).data.grid()

,v
0,5905


In [104]:
(df.cols
        .get('v')
        .groupby('g')
        .agg([F.countDistinct, F.approx_count_distinct])
).data.grid()

,g,v_countDistinct,v_approx_count_distinct
0,1,1765,1748
1,3,1835,1892
2,4,1787,1902
3,2,1881,1775
4,0,1828,1807


In [105]:
# using hll_init (less efficient, might OOM on large collections)

from datafaucet.spark import functions as FF
(df
    .cols.get('v').hll_init()
    .cols.get('v').groupby('g').agg(FF.hll_merge())
    .cols.get('v').hll_count()
).data.grid(5)

,g,v
0,1,1794
1,3,1806
2,4,1755
3,2,1850
4,0,1805


In [106]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark import functions as FF
(df
    .cols.get('v').groupby('g').agg(FF.hll_init_agg())
    .cols.get('v').hll_count()
).data.grid()

,g,v
0,1,1794
1,3,1806
2,4,1755
3,2,1850
4,0,1805


In [107]:
# using hll_init_agg (very efficient: sketch in memory)

from datafaucet.spark.functions import hll_count, hll_init_agg
(df
    .groupby('g').agg(hll_init_agg()(F.col('v')).alias('v'))
    .select('g', hll_count()(F.col('v')).alias('count'))
).data.grid()

,g,count
0,1,1794
1,3,1806
2,4,1755
3,2,1850
4,0,1805


In [108]:
#pre-aggregate the cube
df_cube = (df
    .cols.get('v').groupby('g').agg(('v_sketch', hll_init_agg()))
)
cube = df_cube.data.collect()
cube

,g,v
0,1,"[0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 1, 0, 2, 0, 0, ..."
1,3,"[0, 1, 1, 0, 0, 0, 0, 0, 4, 5, 0, 0, 0, 0, 0, ..."
2,4,"[0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,2,"[0, 0, 0, 2, 4, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
4,0,"[0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."


In [109]:
# re-aqggregate on the cube with hll_merge + hll_count
(df_cube
    .cols.get('v').hll_count()
).data.grid()

,g,v
0,1,1794
1,3,1806
2,4,1755
3,2,1850
4,0,1805


In [110]:
# partial count on the cube: filter + hll_merge + hll_count
(df_cube
     .rows.filter("g in(0,2,4)")
     .cols.get('v').agg('hll_merge')
     .cols.get('v').hll_count()
).data.grid()

,v
0,4524


In [111]:
# verify the result with countDistinct, and approx_count_distinct
(df
    .rows.filter("g in (0,2,4)")
    .cols.get('v').agg([F.countDistinct, F.approx_count_distinct])
).data.grid()

,v_countDistinct,v_approx_count_distinct
0,4534,4358


In [112]:
r = range(0,GROUPS)

sel = [[x for x in r]]
sel += [[x for x in r if x%2==i] for i in range(2)]
sel += [[x for x in r if x//3==i] for i in range(2)]
sel

[[0, 1, 2, 3, 4], [0, 2, 4], [1, 3], [0, 1, 2], [3, 4]]

In [113]:
s = [ str(x)[1:-1] for x in sel]
s

['0, 1, 2, 3, 4', '0, 2, 4', '1, 3', '0, 1, 2', '3, 4']

In [114]:
%%time
def count_selection(e):
    return (df_cube
     .rows.filter(f"g in({e})")
     .cols.get('v').agg('hll_merge')
     .cols.get('v').hll_count()
    ).data.collect(1)['v'][0]

import pandas as pd
import functools

res =  [count_selection(e) for e in s]
spark_hll = pd.DataFrame(res, columns=['v_spark_hll'])
spark_hll

CPU times: user 198 ms, sys: 60 ms, total: 258 ms
Wall time: 9.91 s


,v_spark_hll
0,6336
1,4524
2,3251
3,4459
4,3262


In [115]:
%%time
import pandas as pd
from HLL import HyperLogLog

def count_selection(e):
    hll_res = HyperLogLog(k=12)
    hll = HyperLogLog(k=12)
    for i in e:
        hll.set_registers(cube[cube['g']==i]['v'].iloc[0])
        hll_res.merge(hll)
    return int(hll_res.cardinality())

res =  [count_selection(e) for e in sel]
pandas_hll = pd.DataFrame(res, columns=['v_pandas_hll'])
pandas_hll

CPU times: user 56.3 ms, sys: 3.11 ms, total: 59.4 ms
Wall time: 57.6 ms


,v_pandas_hll
0,6336
1,4524
2,3251
3,4459
4,3262


In [117]:
%%time
def count_selection(e):
    return (df
     .rows.filter(f"g in({e})")
     .cols.get('v').agg(F.countDistinct)
    ).data.collect(1)['v'][0]

res =  [count_selection(e) for e in s]
spark_distinct = pd.DataFrame(res, columns=['v_spark_distinct'])

CPU times: user 39.6 ms, sys: 12.2 ms, total: 51.9 ms
Wall time: 2.25 s


In [118]:
d = spark_distinct.join(spark_hll).join(pandas_hll)
d['error_pct'] = 100*(d['v_spark_hll']-d['v_spark_distinct'])/d['v_spark_distinct']
d

,v_spark_distinct,v_spark_hll,v_pandas_hll,error_pct
0,6320,6336,6336,0.253165
1,4534,4524,4524,-0.220556
2,3265,3251,3251,-0.428790
3,4516,4459,4459,-1.262179
4,3316,3262,3262,-1.628468
